## Give me DMI & anatomic segment split by Health Insititions (with geographic details)

In [ ]:
import pandas as pd 

path_input='./../data/data_raw/'
path_output='./../data/data_created/' 

pd.set_option("display.max_columns",None)

### input and output du notebook
- Input
    * Req_HI_Lab_Users.csv
    * Request.csv
    * HealthInstitution.csv
    * FosUserUser.csv
    * OperatingBlock.csv
    * BodyLocation_modif.csv
    * Laboratory.csv
    * Specialty.csv
    * OperatingSpecialty.csv
- Ouput
    * MaterialByHi.csv
    * HiMatGeo.csv
    

In [ ]:
df_hi = pd.read_csv(path_output+"Req_HI_Lab_Users.csv", low_memory=False)
df_hi.head(3)

In [ ]:
df_rq = pd.read_csv(path_input+"Request.csv")
df_lab = pd.read_csv(path_input+"Laboratory.csv")
df_hi = pd.read_csv(path_input+"HealthInstitution.csv")
df_ob = pd.read_csv(path_input+"OperatingBlock.csv")
df_user = pd.read_csv(path_input+"FosUserUser.csv")
df_spec = pd.read_csv(path_input+"Specialty.csv")
df_bodyloc = pd.read_csv(path_input+"BodyLocation_modif.csv")
df_SpeOpe = pd.read_csv(path_input+"OperatingSpecialty.csv")

In [ ]:
df_Spe = pd.merge(df_rq, df_SpeOpe, left_on = "Specialty ID", right_on = "ID" )
df_Spe.head(3)

In [ ]:
df_operation = pd.merge(df_Spe, df_bodyloc, left_on = "Location ID", right_on = "ID" )
df_operation.head(2)

### `df_operation` : tables requête, materiel (specialty), segment anatomique (body location)

In [ ]:
df_operation = df_operation[["ID_x","Name_x","SegmentAnat"]]
df_operation = df_operation.rename(columns = {"ID_x":"Id_rq","Name_x" : "Material"})
df_operation.head(3)

### Columns selection

In [ ]:
#Request
df = df_rq[['ID', 'Chosen Laboratory ID', "Specialty ID",
            'Location ID', 'Operating Block User ID']]
#Labos 
df_lab = df_lab[['ID','Name','Bloc ID']]

#HI
df_hi=df_hi[["ID","Name","City","Zip Code"]]

#OpeBlock
df_ob = df_ob[["ID", "Name", "City", "Health Institution ID"]]

#Users
df_user = df_user[["ID","Laboratory ID", "Operating Block ID", "Surgeon"]]

In [ ]:
df_ob.head(3)

In [ ]:
df = df.merge(df_lab, left_on="Chosen Laboratory ID", right_on="ID")
df = df.rename(columns={'ID_x': 'ID_rq', 'ID_y': 'ID_lab',"Name":"Name_labo"})
df.head(4)

### Merging

In [ ]:
#Labos 
df = df.merge(df_lab, left_on="Chosen Laboratory ID", right_on="ID")
df = df.rename(columns={'ID_x': 'ID_rq', 'ID_y': 'ID_lab'})

#Users
df = df.merge(df_user, left_on="Operating Block User ID", right_on="ID")
df = df.rename(columns={'ID': 'ID_user'})

#OpeBlock
df = df.merge(df_ob, left_on="Operating Block ID", right_on="ID")
df = df.rename(columns={'Name_x': 'Name_lab',
                        'Name_y': 'Name_ob', 'ID': 'ID_ob'})

#HI
df = df.merge(df_hi, left_on="Health Institution ID", right_on="ID")
df = df.rename(columns={'City_x': 'City_ob', 'Zip Code_x': 'Zip Code_ob',
                        'City_y': 'City_hi', 'Zip Code_y': 'Zip Code_hi',
                        'Name': 'Name_hi', 'ID': 'ID_hi'})


In [ ]:
df.head(3)

In [ ]:
df_all = df.merge(df_operation, left_on = "ID_rq",right_on = "Id_rq")
df_all.head(2)

In [ ]:
df_all = df_all[df_all["Name_lab"] != "TEST"]
df_all.head(3)

# HiMatGeo.csv
ID_rq, Name_hi, City_hi, Zip Code, Material, SegmentAnat

### `df_HiMat` =>> df HI & material 

In [ ]:
df_HiMat = df_all [["ID_rq","Name_hi","City_hi","Zip Code","Material","SegmentAnat"]]
df_HiMat.sample(5)

In [ ]:
df_HiMat = df_HiMat [df_HiMat["Name_hi"] != "Centre hospitalier du Pélican"]
df_HiMat.sample(3)

In [ ]:
df_HiMat.to_csv(path_output+"HiMatGeo.csv")

# MaterialByHi.csv

In [ ]:
df_materialbyHIBIS = df_HiMat.groupby(["Material","Name_hi"])["ID_rq"].count().sort_values(ascending=False).to_frame()
df_materialbyHIBIS

In [ ]:
df_merge = df_HiMat.merge(df_materialbyHIBIS, on = "Name_hi")
df_merge.head()

In [ ]:
df_materialbyHIBIS.to_csv(path_output+"MaterialByHi.csv")

### Nb requetes par ES : 

In [ ]:
df_RqbyHI = df_HiMat.groupby('Name_hi')["ID_rq"].count().sort_values(ascending=False).to_frame()
df_RqbyHI.head()